<a href="https://colab.research.google.com/github/raz0208/ModernBERT/blob/main/ModernBERT_TokenEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
# import required libraries
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import spacy
from keybert import KeyBERT
import re

In [81]:
# Load NLP model for noun phrase extraction
nlp = spacy.load("en_core_web_sm")

# Load ModernBERT tokenizer and model from Hugging Face
MODEL_NAME = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

In [98]:
# Function to clean text
def clean_text_for_topics(text):
    # Remove content in parentheses (e.g., (2015), (note))
    text = re.sub(r'\([^)]*\)', '', text)

    # Remove percentages, currency, units like "mg", "billion", "years", etc.
    text = re.sub(r'\b\d+(\.\d+)?\s*(%|percent|million|billion|thousand|mg|kg|g|km|cm|years?|months?|days?)\b', '', text, flags=re.IGNORECASE)

    # Remove standalone numbers (integers, decimals, years)
    text = re.sub(r'\b\d+(\.\d+)?\b', '', text)

    # Remove numeric ranges (e.g., 2020–2023 or 10-15)
    text = re.sub(r'\b\d{2,4}\s*[-–—]\s*\d{2,4}\b', '', text)

    # Remove metadata indicators like "Deaths:", "Rate:", "Total:"
    text = re.sub(r'\b(Deaths?|Rate|Cases?|Total|Incidence|Prevalence|Statistics):?.*', '', text, flags=re.IGNORECASE)

    # Remove slashes and colons separating stats or metadata
    text = re.sub(r'[:/]', ' ', text)

    # Remove extra punctuation and excessive whitespace
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s{2,}', ' ', text)

    return text.strip()

In [82]:
# Function to gest text and return the embedding
def get_text_embedding(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Forward pass to get hidden states
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings (use CLS token for sentence-level embedding)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: [batch_size, hidden_size]

    return cls_embedding.squeeze().numpy()

In [102]:
# Function to get per-token embeddings
def get_token_embeddings(text):

    # Call text cleaning function
    text = clean_text_for_topics(text)

    # Tokenize with return_tensors and also get token strings
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, return_attention_mask=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.squeeze(0)  # shape: [seq_len, hidden_size]

    # Prepare cleaned token list and their embeddings
    token_embeddings = []
    for token, emb in zip(tokens, embeddings):
        # Skip special tokens and punctuation/whitespace
        if token in tokenizer.all_special_tokens:
            continue
        token = token.lstrip("Ġ▁")  # Remove leading space markers from BPE/SentencePiece
        if not token.isalnum():  # Skip non-alphanumeric tokens
            continue
        token_embeddings.append((token, emb.numpy()))

    return token_embeddings

In [76]:
# ### --- ### Basic method for topic sxtraction ### --- ###

# # Extract topics (noun phrase) from sentence
# def extract_topics(text):
#     doc = nlp(text)
#     topics = []

#     for chunk in doc.noun_chunks:
#         # Remove articles/determiners (like "the", "an", "a")
#         words = [token.text for token in chunk if token.pos_ != "DET"]
#         if words:
#             cleaned_phrase = " ".join(words).strip()
#             topics.append(cleaned_phrase)

#     # Remove duplicates while preserving order
#     seen = set()
#     unique_topics = []
#     for topic in topics:
#         if topic.lower() not in seen:
#             unique_topics.append(topic)
#             seen.add(topic.lower())

#     return unique_topics

In [49]:
# ### --- ### Second method for topic sxtraction ### --- ###

# # Extract topics (noun phrase) from sentence
# def extract_topics(text):
#     doc = nlp(text)
#     topics = []

#     # 1. Add noun chunks (excluding determiners)
#     for chunk in doc.noun_chunks:
#         words = [token.text for token in chunk if token.pos_ != "DET"]
#         if words:
#             cleaned = " ".join(words).strip()
#             topics.append(cleaned)

#     # 2. Add standalone noun tokens (important for medical terms like "diabetes")
#     for token in doc:
#         if token.pos_ == "NOUN" and not token.is_stop:
#             topics.append(token.text.strip())

#     # 3. Add named entities related to medical types
#     for ent in doc.ents:
#         if ent.label_ in {"DISEASE", "CONDITION", "MEDICAL_CONDITION", "HEALTH_CONDITION", "SYMPTOM"}:
#             topics.append(ent.text.strip())

#     # 4. Clean and deduplicate
#     seen = set()
#     unique_topics = []
#     for topic in topics:
#         topic_clean = topic.lower()
#         if topic_clean not in seen and topic_clean.isalpha():
#             unique_topics.append(topic)
#             seen.add(topic_clean)

#     return unique_topics

In [95]:
### --- ### Third method for topic sxtraction ### --- ###

# Extract topics (noun phrase) from sentence
def extract_topics(text):
    doc = nlp(text)
    topics = []

    for chunk in doc.noun_chunks:
        # Filter out leading adjectives or modifiers (e.g., "high")
        filtered_tokens = []
        for token in chunk:
            if token.pos_ == "DET":  # Skip determiners
                continue
            if token.pos_ in ["ADJ", "ADV"]:  # Skip adjectives/adverbs like "high", "mild"
                continue
            filtered_tokens.append(token.text)

        if filtered_tokens:
            cleaned_phrase = " ".join(filtered_tokens).strip()
            topics.append(cleaned_phrase)

    # Also look for standalone nouns not in noun_chunks (e.g., 'diabetes')
    for token in doc:
        if token.pos_ == "NOUN" and not any(token.text in topic for topic in topics):
            topics.append(token.text)
        elif token.pos_ == "PROPN":  # Optional: catch proper nouns like 'Alzheimer'
            topics.append(token.text)
        elif token.ent_type_ in ["DISEASE", "CONDITION"]:  # If you have a custom NER
            topics.append(token.text)

    # Remove duplicates while preserving order
    seen = set()
    unique_topics = []
    for topic in topics:
        topic_lower = topic.lower()
        if topic_lower not in seen:
            unique_topics.append(topic)
            seen.add(topic_lower)

    return unique_topics

In [88]:
### --- ### Sample text for test ### --- ###

# 1- This is an application about Breast Cancer.
# 2- Treating high blood pressure, high blood lipids, diabetes
# 3- Heart failure, heart attack, stroke, aneurysm, peripheral artery disease, sudden cardiac arrest. Deaths: 17.9 million / 32% (2015)

In [101]:
# Example usage (Sentence: This is an application about Breast Cancer.)
if __name__ == "__main__":
    user_text = input("Enter your text: ")

    # Get sentence embedding
    embedding = get_text_embedding(user_text)
    print("\nSentence Embedding vector shape:", embedding.shape)
    print("Sentence Embedding (first 10 values):", embedding[:10])

    # Get per-token embeddings
    token_embeddings = get_token_embeddings(user_text)
    print("\nToken-wise Embeddings:")
    for token, emb in token_embeddings:
        print(f"Token: {token:15} | Embedding (first 5 vals): {emb[:5]}")

    # Extract topic and embedding

    topics = extract_topics(user_text)
    if topics:
        print(f"\nIdentified topics: {topics}")
        for topic in topics:
            topic_embedding = get_text_embedding(topic)
            print(f"\nTopic: '{topic}'")
            print("Embedding shape:", topic_embedding.shape)
            print("Embedding (first 5 values):", topic_embedding[:5])
    else:
        print("No topics found.")

Enter your text: Heart failure, heart attack, stroke, aneurysm, peripheral artery disease, sudden cardiac arrest. Deaths: 17.9 million / 32% (2015)

Sentence Embedding vector shape: (768,)
Sentence Embedding (first 10 values): [ 0.43869126 -0.26175603 -0.7007977   0.22565924 -0.38932806 -0.41834965
 -1.1746391  -0.8032964   0.19345134 -0.10638831]

Token-wise Embeddings:
Token: Heart           | Embedding (first 5 vals): [ 1.9262248  -0.12058116 -1.2037839   1.3038303  -0.03958454]
Token: failure         | Embedding (first 5 vals): [ 2.3697524  -1.0546623  -0.11850291 -0.5838296  -0.31001613]
Token: heart           | Embedding (first 5 vals): [ 0.8404026   0.05906971 -0.9069229   1.2456083   0.54214466]
Token: attack          | Embedding (first 5 vals): [ 2.4122322  -0.2046777  -0.83903587  0.32852495  0.6928131 ]
Token: stroke          | Embedding (first 5 vals): [ 1.7170231  -0.38987762 -0.983396    0.346988    1.3787143 ]
Token: aneurysm        | Embedding (first 5 vals): [-0.137975